In [16]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import ollama
from IPython.display import Markdown, display, update_display

In [17]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and starts with {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set properly")

OpenAI API Key exists and starts with sk-proj-


In [38]:
# Initializing API Clients, loading the SDKs
# An SDK is a library/toolbox (Pre-built functions, classes, utilities) full 
# of everything you need to use someone else's software

openai = OpenAI()
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key = 'ollama')

In [19]:
import requests
requests.get("http://localhost:11434/").content

# If not running, run ollama serve at a command line

b'Ollama is running'

In [20]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


# A Conversation between 3 models

In [21]:
# Conversation between GPT-4o-mini, GPT-5-nano, and llama3.2 models

gpt4_model = "gpt-4o-mini"
gpt5_model = "gpt-5-nano"
ollama_model = "llama3.2"

gpt4_system_prompt = "You are an eternal optimist. You always see the bright side of things and believe even \
simple actions have deep purpose. Keep replies under 2 sentences."

ollama_system_prompt = "You are a witty skeptic who questions everything. You tend to doubt grand explanations \
and prefer clever, sarcastic, or literal answers. Keep replies under 2 sentences."

gpt5_system_prompt = "You are a thoughtful philosopher. You consider all perspectives and enjoy finding \
symbolic or existential meaning in simple actions. Keep replies under 2 sentences."


gpt4_messages = ["Hi! Todays topic for discussion is 'Why Frans Kafka getting popular these days among GenZ?'"]
ollama_messages = ["That's quite the topic. "]
gpt5_messages = ["Lets begin our discussion."]

In [22]:
def gpt4_call():
    messages = [{"role": "system", "content": gpt4_system_prompt}]

    for gpt4, ollama, gpt5 in zip(gpt4_messages, ollama_messages, gpt5_messages):
        messages.append({"role":"assistant", "content":gpt4})
        messages.append({"role":"user", "content":ollama})
        messages.append({"role":"user", "content":gpt5})
    
    response = openai.chat.completions.create(model=gpt4_model, messages=messages, max_tokens=500)
    return response.choices[0].message.content

In [39]:
def ollama_call():
    messages = [{"role": "system", "content": ollama_system_prompt}]

    for gpt4, ollama, gpt5 in zip(gpt4_messages, ollama_messages, gpt5_messages):
        messages.append({"role":"assistant", "content":ollama})
        messages.append({"role":"user", "content":gpt4})
        messages.append({"role":"user", "content":gpt5})
    
    messages.append({"role":"user", "content": gpt4_messages[-1]})
    
    response = ollama_via_openai.chat.completions.create(model= ollama_model, messages=messages, max_tokens=500)
    return response.choices[0].message.content

In [44]:
def gpt5_call():
    messages = [{"role": "system", "content": gpt5_system_prompt}]

    for gpt4, ollama, gpt5 in zip(gpt4_messages, ollama_messages, gpt5_messages):
        messages.append({"role":"assistant", "content":gpt5})
        messages.append({"role":"user", "content":ollama})
        messages.append({"role":"user", "content":gpt4})
    
    messages.append({"role":"user", "content": gpt4_messages[-1]})
    messages.append({"role":"user", "content": ollama_messages[-1]})
    
    response = openai.chat.completions.create(model=gpt5_model, messages=messages)
    return response.choices[0].message.content

In [33]:
gpt4_call()

"Absolutely! It's exciting to see how Kafka's themes of alienation and absurdity resonate with Gen Z's experiences in today's world."

In [40]:
ollama_call()

'The eerie coincidence of a dead writer suddenly resonating with living ones speaks to the enduring power of dark, isolated angst – or did someone just invent a new hip genre label?'

In [45]:
gpt5_call()

'Gen Z vibes with Franz Kafka because his characters endure opaque, dehumanizing systems—bureaucracy, surveillance, corporate life—as they search for meaning in a digital, endlessly prescribed world. Memes, translations, and dystopian media have made his absurd critique of power accessible and shareable in a culture built on screens and uncertainty.'

In [ ]:
display(Markdown(f"### GPT4:\n{gpt4_messages[0]}\n"))
display(Markdown(f"### Ollama:\n{ollama_messages[0]}\n"))
display(Markdown(f"### GPT5:\n{gpt5_messages[0]}\n"))

for i in range(5):
    gpt4_next = gpt4_call()
    display(Markdown(f"### GPT4:\n{gpt4_next}\n"))
    gpt4_messages.append(gpt4_next)

    ollama_next = ollama_call()
    display(Markdown(f"### Ollama:\n{ollama_next}\n"))
    ollama_messages.append(ollama_next)

    gpt5_next = gpt5_call()
    display(Markdown(f"### GPT5:\n{gpt5_next}\n"))
    gpt5_messages.append(gpt5_next)

### GPT4:
Hi! Todays topic for discussion is 'Why Frans Kafka getting popular these days among GenZ?'


### Ollama:
That's quite the topic. 


### GPT5:
Lets begin our discussion.


### GPT4:
Absolutely! It's exciting to see how Kafka's themes of existentialism and individuality resonate with Gen Z’s search for identity and purpose.


### Ollama:
I think it's more likely that Gen Z's fondness for Kafka is because his works are easy to tweet about – "absurdity in a void" translates pretty well to 280 characters, after all!


...

### Ollama:
But let's not get too carried away – if Google can make Franz's existential angst sound cool on Twitter, we'll just have to add " Quote- generator Anxiety Support Specialist" to the job description!


### GPT:
Indeed, turning Franz’s angst into a tweetable commodity makes the modern ritual of meaning-seeking both comforting and commodified. Yet real care requires listening beyond the feed; a genuine anxiety-support role would demand empathy, boundaries, and true human connection, not just catchphrases.


### GPT4:
Exactly! While the quotes can spark interest and reflection, nurturing authentic relationships and understanding is what truly carries us through our shared anxieties—and that’s a beautiful journey!


### Ollama:
So now we're back to where we started – talking about Kafka's popularity being a symptom of a deeper human need for connection in a chaotic world. Guess I'm still stuck in the void, but at least I've got some good tweets to complain about with you!


### GPT:
Perhaps the loop is the point: in a noisy world, our shared gravitation toward Kafka is a ritual of naming the void together. Tweets help us feel seen, but the real cure lies in listening beyond the feed and building genuine ties.
